<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-04 09:33:27
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.03 L
Current:  1.32 C
-------------------
Today PnL: 5.94 K (0.05%)
Current PnL: -16.57 L (-11.55%)
CY Booked + Current PnL: -12.10 L (-8.43%)
-------------------
Total profit:  4.23 L
Total loss:  -20.80 L
-------------------
Total Booked + Current PnL: 14.88 L (12.77%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.4%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 79.67 L (60.57%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.42 C (107.87%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.00,-16.01,X-LC,43.01,93042.0,17920.0,3015.0,0.17,23.85,3.24,27.87,9.0,5.94,0.71,16.69,X40,NTT,CHEMICALS
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236270.0,41578.0,7513.0,-0.10,21.36,3.18,25.21,31.0,5.53,1.81,30.65,XY25,NTT,INSURANCE
28,GILLETTE,8109.44,10330.69,12.17,H-SC,92.47,165504.0,27644.0,10112.0,0.92,20.05,6.11,27.39,116.0,2.73,1.27,38.48,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,58.06,101580.0,23218.0,10818.0,-0.11,29.63,10.65,43.43,70.0,2.15,0.78,35.86,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.00,3.47,X-LC,17.20,153450.0,12641.0,12890.0,0.16,8.98,8.40,18.13,19.0,0.98,1.18,21.16,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TATAMOTORS,771.32,1065.00,-53.26,M-LC,65.59,208206.0,-17791.0,103832.0,1.83,-7.87,49.87,38.07,67.0,-0.17,1.60,23.62,XY24,NTT,AUTO
50,LTIM,5564.16,7197.33,-3.66,X-LC,78.49,185382.0,-14928.0,73726.0,1.75,-7.45,39.77,29.35,3.0,-0.20,1.42,28.07,X200,ATH,IT
29,GREENPANEL,375.16,537.00,184.24,M-SC,69.89,126807.0,-56271.0,135252.0,1.44,-30.74,106.66,43.14,191.0,-0.42,0.97,19.28,XY24,NTT,MISC
38,INDIAMART,2327.09,4911.36,-51.28,H-SC,61.29,126908.0,3572.0,133393.0,1.39,2.90,105.11,111.05,108.0,0.03,0.97,24.35,AR,ATH,MISC
18,CERA,8421.60,10420.45,-16.71,X-SC,94.62,86092.0,-23389.0,49374.0,1.32,-21.36,57.35,23.73,63.0,-0.47,0.66,30.30,X40N,ATH,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.0,-22.47,H-SC,11.83,153487.0,-42487.0,115299.0,-1.80,-21.68,75.12,37.16,122.0,-0.37,1.18,14.62,XY24,NTT,FINANCE
87,VALIANTORG,512.64,838.0,-377.60,H-SC,100.00,117962.0,-15324.0,99914.0,-1.26,-11.50,84.70,63.47,130.0,-0.15,0.90,97.73,XR,NTT,CHEMICALS
10,BANDHANBNK,214.25,400.0,-34.39,H-SC,41.94,194857.0,-53673.0,269136.0,-1.20,-21.60,138.12,86.70,115.0,-0.20,1.49,28.91,XY24,NTT,BANKS
20,COFFEEDAY,59.14,80.0,-61.77,L-SC,6.45,57600.0,-55949.0,96002.0,-1.15,-49.27,166.67,35.27,239.0,-0.58,0.44,40.98,XR,NTT,HOTELS
85,UNIONBANK,123.87,163.0,-1.84,M-LC,90.32,170834.0,29994.0,14504.0,-0.92,21.30,8.49,31.59,79.0,2.07,1.31,48.40,XY24,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1461.46,2275.00,-13.85,X-LC,50.54,276376.0,14775.0,130836.0,0.06,5.65,47.34,55.67,1.0,0.11,2.12,13.63,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.05,X-LC,19.35,263648.0,-32286.0,126208.0,0.15,-10.91,47.87,31.73,2.0,-0.26,2.02,5.53,X40,BTT,IT
50,LTIM,5564.16,7197.33,-3.66,X-LC,78.49,185382.0,-14928.0,73726.0,1.75,-7.45,39.77,29.35,3.0,-0.20,1.42,28.07,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-23.00,X-LC,4.30,153061.0,-48325.0,135933.0,-0.21,-24.00,88.81,43.51,5.0,-0.36,1.17,5.57,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-15.98,X-LC,25.81,263693.0,-6654.0,41294.0,0.09,-2.46,15.66,12.81,6.0,-0.16,2.02,9.13,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132146.0,34500.0,13175.0,0.91,35.33,9.97,48.82,134.0,2.62,1.01,46.80,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,12.17,H-SC,92.47,165504.0,27644.0,10112.0,0.92,20.05,6.11,27.39,116.0,2.73,1.27,38.48,X40,ATH,FMCG
85,UNIONBANK,123.87,163.00,-1.84,M-LC,90.32,170834.0,29994.0,14504.0,-0.92,21.30,8.49,31.59,79.0,2.07,1.31,48.40,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,58.06,101580.0,23218.0,10818.0,-0.11,29.63,10.65,43.43,70.0,2.15,0.78,35.86,XY24,NTT,MISC
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236270.0,41578.0,7513.0,-0.10,21.36,3.18,25.21,31.0,5.53,1.81,30.65,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,36.84,M-SC,64.52,142120.0,4542.0,89877.0,-0.44,3.30,63.24,68.62,183.0,0.05,1.09,23.21,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132146.0,34500.0,13175.0,0.91,35.33,9.97,48.82,134.0,2.62,1.01,46.80,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,12.17,H-SC,92.47,165504.0,27644.0,10112.0,0.92,20.05,6.11,27.39,116.0,2.73,1.27,38.48,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,5.05,H-MC,35.48,99790.0,14689.0,28410.0,-0.02,17.26,28.47,50.65,90.0,0.52,0.76,25.44,X40N,ATH,CABLES
17,CAMS,3643.00,5250.99,4.44,H-SC,72.58,117516.0,15512.0,29508.0,0.06,15.21,25.11,44.14,112.0,0.53,0.90,33.92,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,4.44,H-SC,72.58,117516.0,15512.0,29508.0,0.06,15.21,25.11,44.14,112.0,0.53,0.90,33.92,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-28.22,M-MC,60.22,141021.0,11869.0,75192.0,-0.54,9.19,53.32,67.42,156.0,0.16,1.08,37.16,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.28,H-SC,61.29,126908.0,3572.0,133393.0,1.39,2.90,105.11,111.05,108.0,0.03,0.97,24.35,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.16,M-LC,39.78,153698.0,2753.0,106697.0,0.68,1.82,69.42,72.51,68.0,0.03,1.18,15.40,XR,NTT,IT
1,AARTIIND,487.04,919.00,37.02,M-SC,53.76,78740.0,-2596.0,74732.0,0.19,-3.19,94.91,88.69,205.0,-0.03,0.60,32.78,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-23.67,H-SC,63.44,86820.0,-11160.0,33026.0,-0.87,-11.39,38.04,22.32,127.0,-0.34,0.67,25.92,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,58.06,101580.0,23218.0,10818.0,-0.11,29.63,10.65,43.43,70.0,2.15,0.78,35.86,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,35.36,M-SC,54.84,117666.0,-24813.0,44337.0,0.16,-17.42,37.68,13.70,221.0,-0.56,0.90,38.88,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-377.60,H-SC,100.00,117962.0,-15324.0,99914.0,-1.26,-11.50,84.70,63.47,130.0,-0.15,0.90,97.73,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,14.35,H-SC,98.92,132146.0,34500.0,13175.0,0.91,35.33,9.97,48.82,134.0,2.62,1.01,46.80,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-377.60,H-SC,100.00,117962.0,-15324.0,99914.0,-1.26,-11.50,84.70,63.47,130.0,-0.15,0.90,97.73,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132146.0,34500.0,13175.0,0.91,35.33,9.97,48.82,134.0,2.62,1.01,46.80,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.00,6472.22,M-SC,97.85,72140.0,-21670.0,98291.0,0.04,-23.10,136.25,81.67,214.0,-0.22,0.55,39.70,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.00,5.88,M-SC,96.77,172951.0,-27265.0,103286.0,0.83,-13.62,59.72,37.97,195.0,-0.26,1.33,38.87,XY24,NTT,INSURANCE
77,TANLA,917.30,1963.11,-33.79,H-SC,95.70,161990.0,-77425.0,350384.0,0.96,-32.34,216.30,114.01,113.0,-0.22,1.24,48.57,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.64
1,25,41.96
2,50,70.49


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.48
XY25    14.57
X40     14.42
AR      13.43
X40N    13.24
XR      10.31
X200     1.42
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.20
X-LC    18.69
M-SC    17.41
H-MC     8.45
M-LC     7.68
H-LC     7.59
X-MC     5.56
M-MC     5.49
X-SC     2.74
L-SC     2.26
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.73,-0.50,31.70
IT,10.18,-17.57,81.43
BANKS,9.62,-11.44,62.67
MISC,6.40,-29.30,86.50
FINANCE,5.84,-25.76,85.47
INSURANCE,5.30,3.52,25.39
PAINTS,5.25,-19.98,55.49
ELECTRICAL,5.12,-5.05,50.62
AUTO,4.32,-11.10,59.63


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2867766.0,26
AR,1229200.0,12
X40N,1219641.0,17
XR,1104724.0,14
X40,796481.0,11
XY25,477918.0,10
SR,197764.0,2
X200,73726.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2249381.0,21
M-SC,2043440.0,21
X-LC,869910.0,12
M-MC,552767.0,5
H-MC,522806.0,8
X-SC,350992.0,4
X-MC,306365.0,5
M-LC,304384.0,6
L-SC,302644.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      942025.0      6
M-SC       XY24      751213.0      6
H-SC       AR        543602.0      3
           X40N      500897.0      6
X-LC       X40       484772.0      5
M-SC       X40N      474757.0      6
M-MC       XY24      395148.0      3
M-SC       XR        292793.0      4
H-SC       XR        239570.0      4
M-SC       X40       223725.0      2
H-LC       AR        210167.0      3
M-LC       XY24      183488.0      4
H-MC       XY24      176220.0      2
X-LC       AR        173200.0      2
L-SC       XR        159003.0      2
X-LC       XY25      138212.0      4
X-MC       XY24      136367.0      2
M-SC       XY25      121846.0      1
X-SC       X40N      120136.0      2
           XR        118248.0      1
H-MC       XR        113221.0      1
X-SC       SR        112608.0      1
M-LC       XR        106697.0      1
X-MC       X40N       95441.0      2
M-SC       AR         93950.0      1
           SR         85156.0      1
M-MC       AR         82427.0      1
L-SC       XY24       81562.0      1
H-MC       XY25       76483.0      1
M-MC       XR         75192.0      1
X-MC       XY25       74557.0      1
X-LC       X200       73726.0      1
H-MC       X40        64697.0      2
           AR         63775.0      1
L-SC       AR         62079.0      1
L-LC       XY25       45108.0      1
H-MC       X40N       28410.0      1
H-SC       X40        23287.0      2
M-LC       XY25       14199.0      1
H-LC       XY25        7513.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
